## Complex  Bayes Deep Learning

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator, BDeuScore
from pgmpy.estimators import HillClimbSearch


data  = pd.read_csv("symptoms.csv")

# Use HillClimbSearch to learn the structure
hc = HillClimbSearch(data)
best_model = hc.estimate()

# Get the learned structure as edges
learned_structure1 = list(best_model.edges())
# print(learned_structure1)


# Create a list of learned structures
learned_structures = [learned_structure1]

# Convert data to string
combined_data = data.astype(str)

# Learn and plot Bayesian network structures
for i, learned_structure in enumerate(learned_structures, start=1):
    # Learn Bayesian network structure using the combined data
    bn_model = BayesianNetwork(learned_structure)
    bn_model.fit(data=combined_data, estimator=BayesianEstimator, prior_type='BDeu')  # 'BDeu' (Bayesian Dirichlet equivalent uniform)

    # Visualize the Bayesian network structure
    fig, ax = plt.subplots(figsize=(10, 9))
    G_bn = nx.DiGraph()
    G_bn.add_edges_from(bn_model.edges)
    G_bn.add_nodes_from(bn_model.nodes)
    pos_bn = nx.circular_layout(G_bn)
    DAG_bn = G_bn.to_directed()
    nx.topological_sort(DAG_bn)

    # Manually adjust the position of 'covid_severity' node
#     pos_bn['covid_severity'] = (pos_bn['covid_severity'][0] * 0.2, pos_bn['covid_severity'][1] * 0.2)

    nx.draw_networkx(
        G_bn,
        pos=pos_bn,
        with_labels=True,
        node_size=7500,
        arrowsize=30,
        alpha=0.7,
        font_weight="bold",
        font_size=13,
        node_color='#76CC7E',
        width=2.0,
        ax=ax
    )

    # Customize circular for 'covid_severity' node
    tt_g = G_bn.subgraph(nodes=['covid_severity'])
    nx.draw(
        tt_g,
        pos=pos_bn,
        with_labels=False,
        arrowsize=0,
        node_size=9500,
        alpha=0.7,
        font_weight="bold",
        font_size=13,
        node_color='#F15241',  # Customize the color here
        ax=ax,
        width=2.0  # Set the width of the arrows here
    )

    # Save the figure as a PDF with a unique filename
    filename = f'deep_complex.pdf'
    plt.savefig(filename, bbox_inches='tight')  # Set bbox_inches to 'tight'
    plt.show()  # Display the plot
    plt.close()
